In [ ]:
from pathlib import Path
import os
import urllib.parse

from dotenv import load_dotenv
import requests

In [ ]:
load_dotenv("../../aws.env")

In [ ]:
api_url = urllib.parse.urljoin(os.environ["DBAPI_URL"],
                               os.environ["DBAPI_STAGE"])

In [ ]:
output_dir = Path("20210418_load_s3_out")
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# request token
login_data_dict = {
  "username": os.environ["FIRST_USER"],
  "password": os.environ["FIRST_USER_PASSWORD"]
}
r = requests.post(f"{api_url}/token", data=login_data_dict)
a_token = r.json()["access_token"]
token_headers = {"Authorization": f"Bearer {a_token}"}

In [ ]:
# request episodes
id_prefixes = ["PythonBytes", "TalkPythonToMe"]
no_ranges = [range(217, 228), range(295, 306)]
for id_prefix, no_range in zip(id_prefixes, no_ranges):
    for episode_no in no_range:
        episode_id = f"{id_prefix}:{episode_no}"
        episode_url = f"{api_url}/documents/{episode_id}"
        r = requests.get(episode_url, headers=token_headers)
        if r.status_code != 200:
            continue
        j = r.json()
        out_file = output_dir / (episode_id.replace(":", "_") + ".txt")
        with open(out_file, "wt") as fout:
            fout.write(j["parsed_text"])